In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import h5py
import scipy.io as sio
import skimage.io

from Preprocess_Utilities import *

In [4]:
# data path
route_path = "../Data/090085/"
road_type = np.array(["motorway", "trunk", "primary", "secondary", "tertiary", "unclassified", "track", # 0-6
                      "residential", "service", "road", "living_street", # 7-10
                      "all_roads"]) # 11 
#                       "motor_trunk_pri_sec_tert_uncl_track", "motor_trunk_pri_sec_tert_uncl"]) # 12-13

path_raw_image = route_path + "090085_20170531.h5"
path_road_mask = np.char.add(np.char.add(np.char.add(route_path+"Road_Data/",road_type),
                                         np.char.add('/', road_type)), '.tif')

In [5]:
# read in raw image
raw_image = np.array(h5py.File(path_raw_image)['scene'])

# read in road mask
road_img_list = []
for cur_path in path_road_mask:
    print(cur_path)
    road_img_list.append(skimage.io.imread(cur_path))

road_img_list = np.array(road_img_list)

# assert 0-1 coding
assert (np.logical_or(road_img_list == 1, road_img_list == 0)).all()

../Data/090085/Road_Data/motorway/motorway.tif
../Data/090085/Road_Data/trunk/trunk.tif
../Data/090085/Road_Data/primary/primary.tif
../Data/090085/Road_Data/secondary/secondary.tif
../Data/090085/Road_Data/tertiary/tertiary.tif
../Data/090085/Road_Data/unclassified/unclassified.tif
../Data/090085/Road_Data/track/track.tif
../Data/090085/Road_Data/residential/residential.tif
../Data/090085/Road_Data/service/service.tif
../Data/090085/Road_Data/road/road.tif
../Data/090085/Road_Data/living_street/living_street.tif
../Data/090085/Road_Data/all_roads/all_roads.tif


In [6]:
print(type(raw_image), raw_image.shape)
print(type(road_img_list), road_img_list.shape)

<class 'numpy.ndarray'> (7, 7961, 8091)
<class 'numpy.ndarray'> (12, 7961, 8091)


In [7]:
# combine the road mask
print("Used labels:")
combined_road_mask = 0
for i in range(5):
    print(path_road_mask[i])
    combined_road_mask += road_img_list[i]
print(combined_road_mask.shape, (combined_road_mask > 1).any())

combined_road_mask[np.where(combined_road_mask > 1)] = 1
assert (np.logical_or(combined_road_mask == 1, combined_road_mask == 0)).all()

Used labels:
../Data/090085/Road_Data/motorway/motorway.tif
../Data/090085/Road_Data/trunk/trunk.tif
../Data/090085/Road_Data/primary/primary.tif
../Data/090085/Road_Data/secondary/secondary.tif
../Data/090085/Road_Data/tertiary/tertiary.tif
(7961, 8091) True


In [ ]:
# save the combined road mask
save_path = "../Data/090085/Road_Data/motor_trunk_pri_sec_tert/motor_trunk_pri_sec_tert.tif"
skimage.io.imsave(save_path, combined_road_mask)

skimg_mask = skimage.io.imread(save_path)
print((skimg_mask == combined_road_mask).all())
assert (np.logical_or(skimg_mask == 1, skimg_mask == 0)).all()

In [9]:
# record the top-left coordinate of each possible patches & calculate mu, sigma
step = 28
topleft_coordinate = []

# get some negative sample near the positive one
row_offset = 0
while (row_offset + step <= raw_image.shape[1]):
    col_offset = 0
    while (col_offset + step <= raw_image.shape[2]):
        cur_img_patch = raw_image[:,row_offset:row_offset+step, col_offset:col_offset+step]
        
        if (cur_img_patch != -9999).all():
            topleft_coordinate.append((row_offset, col_offset))
        
        col_offset += 1
    row_offset += 1

topleft_coordinate = np.array(topleft_coordinate)
print(topleft_coordinate.shape)

(17154396, 2)


In [12]:
# save set
h5_path = "../Data/090085/Road_Data/motor_trunk_pri_sec_tert/topleft_coordinate.h5"
h5f = h5py.File(h5_path, 'w')
h5f.create_dataset(name='topleft_coordinate', shape=topleft_coordinate.shape, data=topleft_coordinate)
h5f.close()

In [13]:
h5f = h5py.File(h5_path, 'r')
h5_topleft_coordinate = np.array(h5f['topleft_coordinate'])
h5f.close()

print((h5_topleft_coordinate == topleft_coordinate).all())

True


In [ ]:
# plot out some patches
# [0-21450]
index = 1000

has_road = road_existence[index]
mask = road_patch[index]
patch = np.array([image_patch[index][3].T, image_patch[index][2].T, image_patch[index][1].T]).T

print(has_road, patch.max(), patch.min(), patch.shape, mask.shape)
patch = patch * 0.0001 * 5

# plot raw img
plt.imshow(patch)
plt.show()
plt.clf()

# plot road mask
plt.imshow(mask, cmap=plt.get_cmap('gray'))
plt.show()
plt.clf()